本章将介绍numpy中的常用函数，主要涉及如下内容：
- 数组相关的函数
- 从文件中载入数据
- 将数组写入文件
- 简单的数学和统计分析函数

In [1]:
import numpy as np

## 1 读写文件

In [3]:
# 使用eye函数创建单位矩阵
i2 = np.eye(2)
i2

array([[1., 0.],
       [0., 1.]])

In [4]:
# 使用savetxt函数将数据存储到文件中，当然我们需要指定文件名以及要保存的数组
np.savetxt('eye.txt', i2)  # 会在当前文件夹下创建eye.txt文件  里面的内容为i2

## 2 CSV文件

Numpy中的loadtxt函数可以方便地读取csv文件，自动切分字段，并将数据宅如numpy数组。

In [5]:
# (6,7)表示data.csv文件最后两行。usecols的参数为一个元组，
# unpack参数设置为True，意思是分拆存储不同列的数据，即分别将收盘价和成交量的数组赋值给变量c和v                                                      
c, v = np.loadtxt('data.csv', delimiter=',', usecols=(6, 7), unpack=True)  

In [6]:
# 计算成交量加权平均价格（VWAP）
vwap = np.average(c, weights=v)   # v作为权重参数使用，完成了vwap的计算  关于加权均值的计算详见：https://blog.csdn.net/sunys312/article/details/79526000
print("vwap = ", vwap)  

vwap =  350.5895493532009


In [7]:
# 算数平均数
print('mean = ', np.mean(c))

mean =  351.0376666666667


In [8]:
# 时间加权平均价格
t = np.arange(len(c))
print('twap = ', np.average(c, weights=t))

twap =  352.4283218390804


<center>突击测验：计算加权平均值</center>

问题1： 以下哪个函数可以返回数组元素的加权平均值？
<br>(1) weighted average
<br>(2) waverage
<br>(3) average
<br>(4)avg

<br>----------------------------------<br>
**答案**：（3）

In [9]:
# 找出最大值和最小值
# min和max函数能够满足需求。可以按照下面步骤来找最大值和最小值
h,l = np.loadtxt('data.csv', delimiter=',', usecols=(4,5), unpack=True)  
print('highest = ', np.max(h))
print('lowest = ', np.min(l))

highest =  364.9
lowest =  333.53


In [10]:
# numpy中还有ptp函数可以计算数组的取值范围。该函数返回的是数组元素的最大值和最小值之间的差值.相当于max() - min()
print("Spread high price", np.ptp(h))
print("Spread low price", np.ptp(l))

Spread high price 24.859999999999957
Spread low price 26.970000000000027


## 3 动手实践：简单统计分析

（1）计算收盘价的中位数。

In [11]:
c = np.loadtxt('data.csv', delimiter=',', usecols=(6,), unpack=True)
print("中位数为：",np.median(c))   # 使用median函数找到中位数

中位数为： 352.055


In [12]:
# 验证上面是否正确
sorted_close = np.msort(c)   # 调用msort()函数对c进行排序(默认升序)
sorted_close

array([336.1 , 338.61, 339.32, 342.62, 342.88, 343.44, 344.32, 345.03,
       346.5 , 346.67, 348.16, 349.31, 350.56, 351.88, 351.99, 352.12,
       352.47, 353.21, 354.54, 355.2 , 355.36, 355.76, 356.85, 358.16,
       358.3 , 359.18, 359.56, 359.9 , 360.  , 363.13])

In [16]:
N = len(c)
print("中位数middle = ", (sorted_close[int(N/2)] + sorted_close[int((N-1)/2)])/2)

中位数middle =  352.055


In [17]:
# 方差可以体现透支风险的大小
print('variance = ', np.var(c))   # 调用var函数计算方差

variance =  50.126517888888884


In [20]:
# 验证var函数额有效性
print('validate the variance = ', np.mean((c - np.mean(c)) ** 2))

validate the variance =  50.126517888888884


## 4 股票收益率

（1）numpy中的diff函数可以返回一个由相邻数组元素的插值构成的数组。这有点类似于微积分中的微分。为了计算收益率，我们还需要用插值初一前一天的价格。不过需要注意，diff返回的数组比收盘价数组少一个元素

In [21]:
returns = np.diff(c) / c[:-1]  # 没有用收盘价数组中的最后一个值做除数，接下来，用std函数计算标准差
print("standard deviation = ", np.std(returns))

standard deviation =  0.012922134436826306


In [22]:
# 对数收益计算起来甚至更简单一些，先用log函数得到每一个收盘价的对数在对结果使用diff函数即可
logreturns = np.diff(np.log(c))

In [23]:
# 对那些交易日的收益率为正值感兴趣，在完成了前面的步骤之后，我们只需要用where函数就可以做到这一点。where函数可以根据指定的条件返回
#所有满足条件的数组元素的索引值
print("Indices with positive returns ", np.where(returns > 0))

Indices with positive returns  (array([ 0,  1,  4,  5,  6,  7,  9, 10, 11, 12, 16, 17, 18, 19, 21, 22, 23,
       25, 28], dtype=int64),)


In [24]:
print("Indices with negative returns ", np.where(returns <= 0))

Indices with negative returns  (array([ 2,  3,  8, 13, 14, 15, 20, 24, 26, 27], dtype=int64),)


In [26]:
# 在投资学中，波动率(volatility)是对价格变动的一种度量。历史波动率可以根据历史价格数据计算得到
annual_volatility = np.std(logreturns)/np.mean(logreturns)
annual_volatility = annual_volatility / np.sqrt(1./252.)
print("annual_volatility = ", annual_volatility / np.sqrt(1./252.))
print("Monthly volatility", annual_volatility * np.sqrt(1./12.))

annual_volatility =  2052.1736693701687
Monthly volatility 37.318417377317765


##  5 日期分析 

In [29]:
import datetime

In [36]:
# 写出转换函数
def datestr2num(s):
    return datetime.datetime.strptime(s.decode('ascii'), "%d-%m-%Y").date().weekday()  # 这里得加.decode('ascii') 不然会报错

In [37]:
# datetime对象被转换为date对象，调用weekday方法返回一个数字，这个数字可以是0到6的整数，0代表星期一，6代表星期天
dates, close = np.loadtxt('data.csv', delimiter=',', usecols=(1, 6), converters={1:datestr2num}, unpack=True)
print('Dates = ', dates)

Dates =  [4. 0. 1. 2. 3. 4. 0. 1. 2. 3. 4. 0. 1. 2. 3. 4. 1. 2. 3. 4. 0. 1. 2. 3.
 4. 0. 1. 2. 3. 4.]


In [38]:
# 创建一个包含5个元素的数组，分别代表一周的5个工作日。数组元素将初始化为0
averages = np.zeros(5)

# 我们对每个工作日计算处平均值存放在averages数组中
for i in range(5):
    indices = np.where(dates == i)
    prices = np.take(close, indices)
    avg = np.mean(prices)
    print('Day', i, "price", prices, "Average", avg)
    averages[i] = avg

Day 0 price [[339.32 351.88 359.18 353.21 355.36]] Average 351.7900000000001
Day 1 price [[345.03 355.2  359.9  338.61 349.31 355.76]] Average 350.63500000000005
Day 2 price [[344.32 358.16 363.13 342.62 352.12 352.47]] Average 352.1366666666666
Day 3 price [[343.44 354.54 358.3  342.88 359.56 346.67]] Average 350.8983333333333
Day 4 price [[336.1  346.5  356.85 350.56 348.16 360.   351.99]] Average 350.0228571428571


In [40]:
#用max和min函数找哪个工作日的平均收盘价是最高的，哪个是最低的
top = np.max(averages)
print('Heighest aveage', top)
print("Top day of week", np.argmax(averages))
bottom = np.min(averages)
print("Lowest averages", bottom)
print("Bottom day of the week", np.argmin(averages))

Heighest aveage 352.1366666666666
Top day of week 2
Lowest averages 350.0228571428571
Bottom day of the week 4


## 6 周汇总

我们将要汇总整个交易周中从周一到周五的所有数据。数据覆盖的时间段内有一个节假日：2月21日是总统纪念日。这天是星期一，美国股市休市，因此在我们的示例数据中没有这一天的数据记录。数据中的第一天为星期五，处理起来不太方便。按照如下步骤来汇总数据

In [49]:
import numpy as np
from datetime import datetime

# 星期一 0
# 星期二 1
# 星期三 2
# 星期四 3
# 星期五 4
# 星期六 5
# 星期天 6
def datestr2num(s):
    return datetime.strptime(s.decode('ascii'), "%d-%m-%Y").date().weekday()

dates, open, high, low, close=np.loadtxt('data.csv', delimiter=',', usecols=(1, 3, 4, 5, 6), converters={1: datestr2num}, unpack=True)
close = close[:16]
dates = dates[:16]

# get first Monday
first_monday = np.ravel(np.where(dates == 0))[0]
print( "The first Monday index is", first_monday)

# get last Friday
last_friday = np.ravel(np.where(dates == 4))[-1]
print ("The last Friday index is", last_friday)

weeks_indices = np.arange(first_monday, last_friday + 1)
print ("Weeks indices initial", weeks_indices)

weeks_indices = np.split(weeks_indices, 3)
print ("Weeks indices after split", weeks_indices)

def summarize(a, o, h, l, c):
    monday_open = o[a[0]]
    week_high = np.max( np.take(h, a) )  # take函数来根据索引值获取数组元素的值
    week_low = np.min( np.take(l, a) )
    friday_close = c[a[-1]]

    return("APPL", monday_open, week_high, week_low, friday_close)

# apply_along_axis函数的介绍可参考：https://www.cnblogs.com/zz22--/p/7498868.html
weeksummary = np.apply_along_axis(summarize, 1, weeks_indices, open, high, low, close)
print ("Week summary", weeksummary)

np.savetxt("weeksummary.csv", weeksummary, delimiter=",", fmt="%s")

The first Monday index is 1
The last Friday index is 15
Weeks indices initial [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
Weeks indices after split [array([1, 2, 3, 4, 5], dtype=int64), array([ 6,  7,  8,  9, 10], dtype=int64), array([11, 12, 13, 14, 15], dtype=int64)]
Week summary [['APPL' '335.8' '346.7' '334.3' '346.5']
 ['APPL' '347.8' '360.0' '347.6' '356.8']
 ['APPL' '356.7' '364.9' '349.5' '350.5']]
